In [37]:
import pandas as pd
import numpy as np
import requests

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium==0.5.0
import folium # map rendering library

In [3]:
CLIENT_ID = "OLGN2BVYYNYFN50D1HIUHJAOJM4UIVUQPJXZPBETELBC1NZH"
CLIENT_SECRET = "Y5EI35ZZRF1BTZ1KEWMXORZO1SW1ROYLGKCTO3IUMTOLTFB1"
VERSION = '20180605' # Foursquare API version

In [65]:
def getNearbyVenues(lat, lng, radius=5000, LIMIT=100):
    
    venues_list=[]
    #for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)

    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=sushi'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng']
    
        ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude'
    ]

    return(nearby_venues)

In [82]:
# url = "https://api.foursquare.com/v2/venues/explore?&client_id=OLGN2BVYYNYFN50D1HIUHJAOJM4UIVUQPJXZPBETELBC1NZH&client_secret=Y5EI35ZZRF1BTZ1KEWMXORZO1SW1ROYLGKCTO3IUMTOLTFB1&v=20180605&ll=43.6542,-79.3606&radius=500&limit=100&query=sushi"

# results = requests.get(url).json()["response"]['groups'][0]['items']
# results
# # for v in results:
# #     print(v['venue']['url'])

In [66]:
df = getNearbyVenues('43.6542','-79.3606')

In [70]:
df.head()

,Venue,Venue Latitude,Venue Longitude
0,JaBistro,43.649687,-79.388090
1,Japango,43.655268,-79.385165
2,Miku,43.641255,-79.377371
3,Yuzu No Hana,43.648301,-79.388336
4,Ki Modern Japanese + Bar,43.647103,-79.379390


In [68]:
map_clusters = folium.Map(location=[43.6542,-79.3606], zoom_start=11)


In [78]:
from sklearn.cluster import KMeans


In [98]:
# set number of clusters
kclusters = 2


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df.drop('Venue', 1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 1, 1, 0, 1, 0, 1], dtype=int32)

In [99]:
u, count = np.unique(kmeans.labels_, return_counts=True)
count

array([39, 61])

In [100]:
toronto_merged = df
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged.head()

,Venue,Venue Latitude,Venue Longitude,Cluster Labels
0,JaBistro,43.649687,-79.388090,0
1,Japango,43.655268,-79.385165,0
2,Miku,43.641255,-79.377371,1
3,Yuzu No Hana,43.648301,-79.388336,0
4,Ki Modern Japanese + Bar,43.647103,-79.379390,1


In [101]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [102]:


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,name, cluster in zip(df['Venue Latitude'], df['Venue Longitude'], df['Venue'], df['Cluster Labels']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters